In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("../data/Global @dvisor April results (answers displayed as text).xlsx")

In [3]:
SSM_ALLOWED = "Same-sex couples should be allowed to marry legally"
SSM_OBTAIN = "Same-sex couples should be allowed to obtain some kind of legal recognition, but not to marry"
SSM_NOT_ALOWED = "Same-sex couples should not be allowed to marry or obtain any kind of legal recognition"

In [4]:
ABORTION_PERMITTED = "Abortion SHOULD be permitted whenever a woman decides she wants one"
ABORTION_PER_CIRCUMSTANCES = ("Abortion SHOULD be permitted in certain circumstances,"
    " such as if a woman has been raped")
ABORTION_ONLY_DANGER = ("Abortion should NOT be permitted under any circumstances,"
    " except when the life of the mother is in danger")
ABORTION_NEVER = "Abortion should NEVER be permitted, no matter what circumstance exists"

In [5]:
to_asterisk = [ "Mexico", "Brazil", "China", "South Africa", "India", "Turkey", "Russia" ]
def append_asterisk(country_name):
    if country_name in to_asterisk:
        return country_name + "*"
    else: return country_name

In [6]:
def classify_age(age):
    if 16 <= age < 35: return "16-34"
    elif 35 <= age < 50: return "35-49"
    elif 50 <= age < 65: return "50-64"
    else: raise ValueError("Age out of range")

In [7]:
survey = pd.DataFrame({
    "respondent_id": df["Respondent_ID"],
    "country": df["QCountry"].apply(append_asterisk),
    "gender": df["A2"],
    "age": df["A3"],
    "age_group": df["A3"].apply(classify_age),
    "marital": df["A7"],
    "weight": df["Weightvar_HF2"],
    "q1": df["HF2_Q1"],
    "q2": df["HF2_Q2"],
    "q3": df["HF2_Q3"],
    "q4": df["HF2_Q4"],
    "q5": df["HF2_Q5"]
})

In [8]:
def get_grouped_pcts(group_cols):
    weights = pd.DataFrame({
        "answer_weight": survey.groupby(group_cols)["weight"].sum() 
    }).reset_index(group_cols[-1]).join(pd.DataFrame({
        "question_weight": survey.groupby(group_cols[:-1])["weight"].sum()
    }))
    pcts = pd.DataFrame({
        group_cols[-1]: weights[group_cols[-1]],
        "pct": (weights["answer_weight"] * 100. / weights["question_weight"]).round(3)
    }).reset_index()
    return pcts

In [9]:
def pct_by_age_group(question, ans_values):
    by_age_group = get_grouped_pcts([ "country", "age_group", question ])
    agg = by_age_group[by_age_group[question].isin(ans_values)]\
        .groupby([ "country", "age_group" ])["pct"].sum().round(1)\
        .unstack("age_group")\
        .fillna(0)
    return agg

## Support for Same-Sex Marriage

The percentages correspond to the proportion of respondents with the view that "Same-sex couples should be allowed to marry legally."

In [10]:
samesex_marriage_by_country = get_grouped_pcts(["country", "q1"])\
    .set_index("q1").ix[SSM_ALLOWED]\
    .set_index("country")["pct"]\
    .round(1)
samesex_marriage_by_country.order(ascending=False)

country
Spain            76.6
Sweden           74.1
Belgium          69.5
Germany          67.7
Great Britain    67.4
Ireland          66.5
Canada           62.3
Argentina        59.4
Mexico*          53.7
Australia        53.4
France           53.0
United States    48.3
Italy            48.2
South Africa*    45.1
Brazil*          40.5
Hungary          30.8
Japan            29.6
China*           29.4
India*           29.2
Turkey*          27.2
South Korea      26.8
Poland           21.2
Russia*          11.4
Name: pct, dtype: float64

## Support for *At Least* Some Legal Recognition of Same-Sex Couples

The percentages correspond to the proportion of respondents with the view that "Same-sex couples should be allowed to marry legally" *or* "Same-sex couples should be allowed to obtain some kind of legal recognition, but not to marry."

In [11]:
samesex_recognition_by_country = get_grouped_pcts(["country", "q1"])\
    .set_index("q1").ix[[SSM_ALLOWED, SSM_OBTAIN]]\
    .groupby("country")["pct"].sum()\
    .round(1)
samesex_recognition_by_country.order(ascending=False)

country
Spain            85.5
Great Britain    84.5
Sweden           82.0
Ireland          81.5
Belgium          78.6
Germany          78.2
France           76.4
Canada           75.2
Argentina        75.2
Italy            75.1
Australia        74.4
Mexico*          69.7
United States    68.8
Brazil*          61.1
China*           58.6
Japan            58.0
South Africa*    57.6
Poland           55.8
Hungary          53.3
South Korea      52.7
India*           47.5
Turkey*          46.3
Russia*          19.9
Name: pct, dtype: float64

## Support for Unrestricted Abortion 

Percentages correspond to the proportion of respondents with the view that "Abortion should be permitted whenever a woman decides she wants one."

In [12]:
abortion_unrestricted_by_country = get_grouped_pcts(["country", "q3"])\
    .set_index("q3").ix[ABORTION_PERMITTED]\
    .groupby("country")["pct"].sum()\
    .round(1)
abortion_unrestricted_by_country.order(ascending=False)

country
Sweden           78.4
France           67.1
Great Britain    66.2
Hungary          64.6
Spain            59.6
Turkey*          58.6
Canada           54.0
Belgium          52.3
Germany          49.8
Australia        49.1
Italy            47.0
United States    39.8
Ireland          37.4
Japan            36.2
Russia*          36.2
India*           34.5
Poland           32.5
China*           31.7
South Korea      30.5
Argentina        24.9
South Africa*    21.2
Mexico*          14.2
Brazil*          13.4
Name: pct, dtype: float64

## Support for Total Abortion *Bans*

Percentages correspond to the proportion of respondents with the view that "Abortion should never be permitted, no matter what circumstance exists."

In [13]:
abortion_never_by_country = get_grouped_pcts(["country", "q3"])\
    .set_index("q3").ix[ABORTION_NEVER]\
    .groupby("country")["pct"].sum()\
    .round(1)
abortion_never_by_country.order(ascending=False)

country
Brazil*          17.7
Mexico*          16.3
South Africa*    12.0
Argentina         9.2
Turkey*           8.0
Poland            7.2
United States     6.2
India*            5.8
Italy             5.0
Ireland           4.4
South Korea       4.1
Russia*           3.7
Canada            3.7
Australia         3.3
Hungary           2.8
China*            2.8
Germany           2.2
Spain             2.0
Japan             1.8
Belgium           1.6
Sweden            1.2
Great Britain     1.0
France            0.6
Name: pct, dtype: float64

In [14]:
pd.DataFrame({
    "samesex_marriage": samesex_marriage_by_country,
    "samesex_recognition": samesex_recognition_by_country,
    "abortion_never": abortion_never_by_country,
    "abortion_unrestricted": abortion_unrestricted_by_country
}).to_csv("../output/country-level-responses.csv", encoding="utf-8")

## Same-Sex Marriage Support by Age Group

In [15]:
ssm_by_age_group = pct_by_age_group("q1", [ SSM_ALLOWED ])
ssm_by_age_group.to_csv("../output/ssm-support-by-age-group.csv", encoding="utf-8")
ssm_by_age_group

age_group,16-34,35-49,50-64
country,,,
Argentina,64.6,60.1,47.0
Australia,67.2,45.5,43.4
Belgium,74.4,66.4,66.7
Brazil*,44.0,34.9,37.5
Canada,71.3,59.8,56.0
China*,43.3,24.5,10.8
France,56.6,55.1,47.2
Germany,75.0,68.8,59.6
Great Britain,75.8,69.0,55.7


## Abortion Proximity and Abortion Views

The "Yes" and "No" answers below correspond to the question, "Have you or has anyone close to you ever had an abortion?" The percentages correspond to the proportion of respondents in each category with the view that "Abortion should be permitted whenever a woman decides she wants one."

In [16]:
def get_proximity_stats_by_group(group_cols=[]):
    x = get_grouped_pcts(group_cols + [ "q4", "q3" ])\
        .set_index([ "q3" ] + group_cols).ix[ABORTION_PERMITTED]\
        .reset_index()\
        .groupby([ "q4" ] + group_cols )["pct"].sum()\
        .round(1)\
        .unstack("q4")
    return x[[ "Yes", "No" ]]

### By proximity and gender

In [17]:
get_proximity_stats_by_group([ "gender" ])

q4,Yes,No
gender,,
Female,55.1,38.7
Male,54.5,37.7


### By proximity and country

In [18]:
unrestricted_abortion_by_proximity = get_proximity_stats_by_group([ "country" ]).sort("Yes", ascending=False)
unrestricted_abortion_by_proximity.to_csv("../output/unrestricted-abortion-by-proximity.csv", encoding="utf-8")
unrestricted_abortion_by_proximity

q4,Yes,No
country,,
Sweden,90.9,66.0
Turkey*,80.5,48.4
Great Britain,75.4,63.2
France,75.3,63.4
Hungary,72.3,58.2
Australia,72.1,37.3
Belgium,71.5,45.5
Spain,67.8,54.9
Germany,65.5,46.1


### By proximity, country, and gender

In [19]:
get_proximity_stats_by_group([ "country", "gender" ])

q4                     Yes    No
country       gender            
Argentina     Female  27.8  28.6
              Male    32.6  13.2
Australia     Female  73.4  41.2
              Male    70.6  33.2
Belgium       Female  66.7  50.6
              Male    78.4  41.0
Brazil*       Female  14.3  11.7
              Male    19.1  11.3
Canada        Female  62.9  45.1
              Male    65.3  56.4
China*        Female  41.9  25.5
              Male    36.5  24.8
France        Female  75.1  62.6
              Male    75.5  64.3
Germany       Female  66.5  47.5
              Male    64.4  44.7
Great Britain Female  78.2  64.4
              Male    71.8  62.1
Hungary       Female  72.1  60.1
              Male    72.5  56.6
India*        Female  48.8  26.6
              Male    42.4  34.9
Ireland       Female  64.9  33.5
              Male    50.5  28.3
Italy         Female  54.3  46.3
              Male    51.3  46.7
Japan         Female  39.4  36.0
              Male    48.7  37.8
Mexico*       Female  19.1  11.5
              Male    24.6   8.3
Poland        Female  64.4  25.0
              Male    59.0  29.1
Russia*       Female  34.9  17.6
              Male    58.7  16.9
South Africa* Female  38.9  14.9
              Male    27.8  14.8
South Korea   Female  39.4  28.2
              Male    43.6  27.1
Spain         Female  70.6  57.7
              Male    64.2  52.6
Sweden        Female  90.8  65.5
              Male    90.9  66.3
Turkey*       Female  83.5  57.1
              Male    70.4  43.6
United States Female  53.5  30.1
              Male    51.0  30.4